# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'09-25-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'09-25-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-09-26 04:23:42,33.93911,67.709953,39186,1451,32619,5116.0,Afghanistan,100.661880,3.702853
1,NaN,NaN,NaN,Albania,2020-09-26 04:23:42,41.15330,20.168300,13045,373,7309,5363.0,Albania,453.297658,2.859333
2,NaN,NaN,NaN,Algeria,2020-09-26 04:23:42,28.03390,1.659600,50754,1707,35654,13393.0,Algeria,115.741831,3.363282
3,NaN,NaN,NaN,Andorra,2020-09-26 04:23:42,42.50630,1.521800,1836,53,1263,520.0,Andorra,2376.237624,2.886710
4,NaN,NaN,NaN,Angola,2020-09-26 04:23:42,-11.20270,17.873900,4590,167,1554,2869.0,Angola,13.965687,3.638344


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,38855,38872,38883,38919,39044,39074,39096,39145,39170,39186
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,11816,11948,12073,12226,12385,12535,12666,12787,12921,13045
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,48966,49194,49413,49623,49826,50023,50214,50400,50579,50754


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1436,1436,1437,1437,1441,1444,1445,1446,1451,1451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,343,347,353,358,362,364,367,370,370,373
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1645,1654,1659,1665,1672,1679,1689,1698,1703,1707


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,32503,32505,32576,32576,32576,32576,32576,32610,32619,32619
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,6733,6788,6831,6888,6940,6995,7042,7139,7239,7309
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,34517,34675,34818,34923,35047,35180,35307,35428,35544,35654


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1619,1624,1664,1673,1690,1691,1714,1715,1738,1757
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,5003,5021,5033,5047,5061,5087,5124,5141,5165,5456
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,809,809,824,830,835,838,848,851,857,873


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,24,24,24,24,24,24,25,25,25,25
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,47,48,48,49,49,49,49,49,50,50
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7,7,7,7,7,7,7,7,7,7


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
632,1001.0,Autauga,Alabama,US,2020-09-26 04:23:42,32.539527,-86.644082,1757,25,0,1732.0,"Autauga, Alabama, US",3144.856718,1.422880
669,1075.0,Lamar,Alabama,US,2020-09-26 04:23:42,33.779950,-88.096680,366,2,0,364.0,"Lamar, Alabama, US",2651.213329,0.546448
670,1077.0,Lauderdale,Alabama,US,2020-09-26 04:23:42,34.901719,-87.656247,1698,33,0,1665.0,"Lauderdale, Alabama, US",1831.142361,1.943463


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,7032712,203750,2727335
India,5818570,92290,4756164
Brazil,4689613,140537,4128490


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,7032712,203750,2727335,4101627,2020-09-26 04:23:42,37.923092,-91.345036
India,5818570,92290,4756164,970116,2020-09-26 04:23:42,23.088275,81.806127
Brazil,4689613,140537,4128490,420586,2020-09-26 04:23:42,-12.669522,-48.480493
Russia,1131088,19973,930955,180160,2020-09-26 04:23:42,54.546312,62.120860
Colombia,798317,25103,687477,85737,2020-09-26 04:23:42,5.349139,-74.190797


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,803660,15516,0
Texas,751068,15610,0
Florida,695887,13915,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,803660,15516,0,788144,2020-09-26 04:23:42,37.843962,-120.728594
Texas,751068,15610,0,735458,2020-09-26 04:23:42,31.660643,-98.653069
Florida,695887,13915,0,681972,2020-09-26 04:23:42,28.940755,-82.700744
New York,453755,33122,0,420633,2020-09-26 04:23:42,42.544151,-75.474183
Georgia,312514,6874,0,305640,2020-09-26 04:23:42,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,265775,6488,0
Florida,Miami-Dade,168775,3202,0
Illinois,Cook,142215,5194,0
Arizona,Maricopa,140753,3343,0
Texas,Harris,140532,2555,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,265775,6488,0,259287,2020-09-26 04:23:42,34.308284,-118.228241,6037.0
Florida,Miami-Dade,168775,3202,0,165573,2020-09-26 04:23:42,25.611236,-80.551706,12086.0
Illinois,Cook,142215,5194,0,137021,2020-09-26 04:23:42,41.841448,-87.816588,17031.0
Arizona,Maricopa,140753,3343,0,137410,2020-09-26 04:23:42,33.348359,-112.491815,4013.0
Texas,Harris,140532,2555,0,137977,2020-09-26 04:23:42,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-23,39145,12787,50400,1753,4363,97,664799,47877,26980,39984,...,412245,1946,53275,69439,1069,37083,10,2029,14443,7725
2020-09-24,39170,12921,50579,1753,4475,97,678266,48251,27000,40816,...,418889,1959,53834,70406,1069,37591,10,2029,14491,7752
2020-09-25,39186,13045,50754,1836,4590,98,691235,48643,27016,41500,...,425767,1967,54392,71273,1069,37963,10,2029,14515,7787


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-23,1446,370,1698,53,159,3,14376,942,861,777,...,41951,47,444,574,35,272,1,586,332,227
2020-09-24,1451,370,1703,53,162,3,14766,945,869,783,...,41991,47,446,581,35,274,1,586,332,227
2020-09-25,1451,373,1707,53,167,3,15208,947,870,786,...,42025,47,449,591,35,278,1,587,332,227


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-23,32610,7139,35428,1203,1473,92,525486,43026,24446,30949,...,2247,1661,49649,58759,991,26288,8,1245,13629,6007
2020-09-24,32619,7239,35544,1203,1503,92,536589,43266,24521,31661,...,2282,1679,50230,59745,991,26934,8,1250,13643,6043
2020-09-25,32619,7309,35654,1263,1554,92,546924,43401,24521,32301,...,2303,1710,50879,60709,999,27183,8,1255,13643,6057


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1619,1624,1664,1673,1690,1691,1714,1715,1738,1757
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,327,330,337,345,346,353,355,357,363,366
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1590,1592,1616,1641,1646,1653,1662,1673,1682,1698
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,573,576,578,582,585,592,603,605,609,616
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,5570,5582,5603,5705,5713,5751,5765,5792,5829,5887


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-23,147153,7004,0,215284,77963,796436,66649,56315,19830,49,...,186798,742913,65921,1722,1290,142459,83702,14515,105932,5169
2020-09-24,148206,7132,0,215852,79049,799516,67196,56472,19947,49,...,187690,747366,67119,1724,1290,143357,84238,14719,108324,5305
2020-09-25,150658,7254,0,216367,79946,803660,67905,56587,20085,49,...,189454,751068,68530,1731,1296,144294,85226,14962,110828,5420


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-09-23        1715    5141     851  638   1580     597    911    3401   
2020-09-24        1738    5165     857  642   1594     598    913    3466   
2020-09-25        1757    5456     873  652   1608     599    916    3499   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-23         1144      583  ...     214     23      262       87   
2020-09-24         1151      591  ...     218     25      270       90   
2020-09-25         1161      603  ...     227     27      272       95   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-23            326   517   342          0      113     25  
2020-09-24            328   529   347          0      113     27  
2020-09-25            332   532   348          0      114     35  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-23,2488,45,0,5525,1229,15291,2030,4497,629,0,...,2277,15372,444,58,19,3087,2081,322,1259,50
2020-09-24,2506,46,0,5559,1246,15393,2033,4499,630,0,...,2312,15510,444,58,19,3111,2080,328,1265,50
2020-09-25,2491,52,0,5587,1266,15516,2034,4501,631,0,...,2352,15610,448,58,19,3134,2100,335,1274,50


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-09-23          25      49       7   10     15      14     39      40   
2020-09-24          25      50       7   10     15      14     39      41   
2020-09-25          25      50       7   10     15      14     39      42   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-23           42       15  ...       2      1        4        1   
2020-09-24           42       15  ...       2      1        4        1   
2020-09-25           42       13  ...       2      1        4        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-23              2     1     2          0        6      0  
2020-09-24              2     1     2          0        6      0  
2020-09-25              2     1     2          0        6      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-23,0.001253,0.009553,0.003704,0.042832,0.029981,0.010417,0.019358,0.004406,0.000297,0.017327,...,0.015237,0.006205,0.011199,0.014404,0.000936,0.013751,0.0,0.000493,0.003753,0.001816
2020-09-24,0.000639,0.010479,0.003552,0.000000,0.025670,0.000000,0.020257,0.007812,0.000741,0.020808,...,0.016117,0.006680,0.010493,0.013926,0.000000,0.013699,0.0,0.000000,0.003323,0.003495
2020-09-25,0.000408,0.009597,0.003460,0.047347,0.025698,0.010309,0.019121,0.008124,0.000593,0.016758,...,0.016420,0.004084,0.010365,0.012314,0.000000,0.009896,0.0,0.000000,0.001656,0.004515


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-23,0.000692,0.008174,0.005329,0.0,0.025806,0.0,0.030390,0.004264,0.002328,0.007782,...,0.000883,0.021739,0.004525,0.017730,0.0,0.011152,0.0,0.000000,0.003021,0.004425
2020-09-24,0.003458,0.000000,0.002945,0.0,0.018868,0.0,0.027129,0.003185,0.009292,0.007722,...,0.000953,0.000000,0.004505,0.012195,0.0,0.007353,0.0,0.000000,0.000000,0.000000
2020-09-25,0.000000,0.008108,0.002349,0.0,0.030864,0.0,0.029934,0.002116,0.001151,0.003831,...,0.000810,0.000000,0.006726,0.017212,0.0,0.014599,0.0,0.001706,0.000000,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-23,0.001044,0.013774,0.003427,0.003336,0.007524,0.0,0.015966,0.008201,0.004809,0.021015,...,0.004021,0.009726,0.011861,0.017049,0.035528,0.032157,0.0,0.004032,0.000000,0.004683
2020-09-24,0.000276,0.014008,0.003274,0.000000,0.020367,0.0,0.021129,0.005578,0.003068,0.023006,...,0.015576,0.010837,0.011702,0.016780,0.000000,0.024574,0.0,0.004016,0.001027,0.005993
2020-09-25,0.000000,0.009670,0.003095,0.049875,0.033932,0.0,0.019261,0.003120,0.000000,0.020214,...,0.009202,0.018463,0.012921,0.016135,0.008073,0.009245,0.0,0.004000,0.000000,0.002317


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-23,0.003882,0.009804,NaN,0.002039,0.012756,0.004251,0.009329,0.002760,0.003492,0.0,...,0.008912,0.001685,0.013483,0.000581,0.009390,0.004053,0.006118,0.008406,0.016915,0.030502
2020-09-24,0.007156,0.018275,NaN,0.002638,0.013930,0.003867,0.008207,0.002788,0.005900,0.0,...,0.004775,0.005994,0.018173,0.001161,0.000000,0.006304,0.006404,0.014054,0.022581,0.026311
2020-09-25,0.016545,0.017106,NaN,0.002386,0.011347,0.005183,0.010551,0.002036,0.006918,0.0,...,0.009398,0.004953,0.021022,0.004060,0.004651,0.006536,0.011729,0.016509,0.023116,0.021678


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-09-23      0.000583  0.003318  0.003538  0.004724  0.004450  0.006745   
2020-09-24      0.013411  0.004668  0.007051  0.006270  0.008861  0.001675   
2020-09-25      0.010932  0.056341  0.018670  0.015576  0.008783  0.001672   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-09-23      0.000000  0.003245  0.003509  0.013913  ...  0.019048   
2020-09-24      0.002195  0.019112  0.006119  0.013722  ...  0.018692   
2020-09-25      0.003286  0.009521  0.008688  0.020305  ...  0.041284   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-09-23      0.095238  0.031496  0.101266   0.009288  0.021739  0.008850   
2020-09-24      0.086957  0.030534  0.034483   0.006135  0.023211  0.014620   
2020-09-25      0.080000  0.007407  0.055556   0.012195  0.005671  0.002882   

Province_State                                
Admin2         Unassigned Washakie    Weston  
2020-09-23            NaN  0.00000 -0.038462  
2020-09-24            NaN  0.00000  0.080000  
2020-09-25            NaN  0.00885  0.296296  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-23,0.012617,0.000000,NaN,0.004911,0.016543,0.006715,0.002469,0.000222,0.001592,NaN,...,0.007077,0.009390,0.002257,0.0,0.0,0.009483,0.005314,0.006250,0.006395,0.020408
2020-09-24,0.007235,0.022222,NaN,0.006154,0.013832,0.006671,0.001478,0.000445,0.001590,NaN,...,0.015371,0.008977,0.000000,0.0,0.0,0.007775,-0.000481,0.018634,0.004766,0.000000
2020-09-25,-0.005986,0.130435,NaN,0.005037,0.016051,0.007991,0.000492,0.000445,0.001587,NaN,...,0.017301,0.006447,0.009009,0.0,0.0,0.007393,0.009615,0.021341,0.007115,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga   Baldwin Barbour Bibb    Blount Bullock    Butler   
2020-09-23         0.0  0.000000     0.0  0.0  0.071429     0.0  0.026316   
2020-09-24         0.0  0.020408     0.0  0.0  0.000000     0.0  0.000000   
2020-09-25         0.0  0.000000     0.0  0.0  0.000000     0.0  0.000000   

Province_State                              ... Wyoming                  \
Admin2          Calhoun Chambers  Cherokee  ...    Park Platte Sheridan   
2020-09-23      0.00000      0.0 -0.062500  ...     0.0    0.0      0.0   
2020-09-24      0.02500      0.0  0.000000  ...     0.0    0.0      0.0   
2020-09-25      0.02439      0.0 -0.133333  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-23          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-09-24          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-09-25          0.0        0.0   0.0   0.0        NaN      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-23,0.001110,0.010466,0.003834,0.031816,0.028743,0.005537,0.018068,0.003820,0.000653,0.017098,...,0.013371,0.005613,0.011236,0.013901,0.000521,0.013887,5.609678e-30,0.000439,0.006600,0.002103
2020-09-24,0.000874,0.010473,0.003693,0.015908,0.027207,0.002769,0.019162,0.005816,0.000697,0.018953,...,0.014744,0.006147,0.010865,0.013913,0.000261,0.013793,2.804839e-30,0.000219,0.004962,0.002799
2020-09-25,0.000641,0.010035,0.003576,0.031628,0.026453,0.006539,0.019142,0.006970,0.000645,0.017856,...,0.015582,0.005115,0.010615,0.013114,0.000130,0.011844,1.402419e-30,0.000110,0.003309,0.003657


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-23,0.000997,0.008408,0.005197,2.186285e-15,0.018861,1.723933e-49,0.030237,0.003731,0.003644,0.005634,...,0.000760,0.011043,0.008204,0.016781,1.896395e-08,0.014122,0.0,0.000135,0.002209,0.003465
2020-09-24,0.002227,0.004204,0.004071,1.093143e-15,0.018864,8.619666e-50,0.028683,0.003458,0.006468,0.006678,...,0.000857,0.005522,0.006354,0.014488,9.481974e-09,0.010738,0.0,0.000068,0.001105,0.001733
2020-09-25,0.001114,0.006156,0.003210,5.465713e-16,0.024864,4.309833e-50,0.029308,0.002787,0.003809,0.005255,...,0.000833,0.002761,0.006540,0.015850,4.740987e-09,0.012668,0.0,0.000887,0.000552,0.000866


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-23,0.000628,0.010524,0.003542,0.007511,0.007079,0.000172,0.020187,0.004741,0.005146,0.020038,...,0.003600,0.008469,0.012424,0.016984,0.021200,0.032072,3.286920e-32,0.004166,0.003296,0.005134
2020-09-24,0.000452,0.012266,0.003408,0.003755,0.013723,0.000086,0.020658,0.005160,0.004107,0.021522,...,0.009588,0.009653,0.012063,0.016882,0.010600,0.028323,1.643460e-32,0.004091,0.002162,0.005563
2020-09-25,0.000226,0.010968,0.003251,0.026815,0.023827,0.000043,0.019959,0.004140,0.002053,0.020868,...,0.009395,0.014058,0.012492,0.016509,0.009336,0.018784,8.217301e-33,0.004046,0.001081,0.003940


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-23,0.004863,0.009494,NaN,0.002255,0.010754,0.004230,0.009263,0.003202,0.004221,1.084658e-58,...,0.007271,0.007776,0.012439,0.001170,0.007367,0.004960,0.005377,0.010805,0.016587,0.022772
2020-09-24,0.006009,0.013885,NaN,0.002447,0.012342,0.004049,0.008735,0.002995,0.005060,5.423292e-59,...,0.006023,0.006885,0.015306,0.001166,0.003683,0.005632,0.005890,0.012430,0.019584,0.024541
2020-09-25,0.011277,0.015495,NaN,0.002416,0.011845,0.004616,0.009643,0.002516,0.005989,2.711646e-59,...,0.007711,0.005919,0.018164,0.002613,0.004167,0.006084,0.008809,0.014470,0.021350,0.023109


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-09-23      0.005428  0.004615  0.007231  0.003949  0.007259  0.005492   
2020-09-24      0.009419  0.004642  0.007141  0.005109  0.008060  0.003584   
2020-09-25      0.010176  0.030491  0.012905  0.010343  0.008421  0.002628   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-09-23      0.000644  0.006010  0.005545  0.013781  ...  0.018034   
2020-09-24      0.001420  0.012561  0.005832  0.013751  ...  0.018363   
2020-09-25      0.002353  0.011041  0.007260  0.017028  ...  0.029824   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-09-23      0.067957  0.025659  0.079018   0.005005  0.014442  0.009068   
2020-09-24      0.077457  0.028097  0.056750   0.005570  0.018826  0.011844   
2020-09-25      0.078728  0.017752  0.056153   0.008883  0.012249  0.007363   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-09-23      -0.571428  0.000210  0.013403  
2020-09-24      -0.571428  0.000105  0.046701  
2020-09-25      -0.571428  0.004477  0.171499  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-23,0.008618,0.000364,NaN,0.003804,0.013198,0.006364,0.002502,0.000269,0.002493,NaN,...,0.008232,0.007313,0.002779,3.141197e-17,0.000015,0.009568,0.006071,0.009726,0.005085,0.010792
2020-09-24,0.007926,0.011293,NaN,0.004979,0.013515,0.006517,0.001990,0.000357,0.002041,NaN,...,0.011801,0.008145,0.001389,1.570599e-17,0.000007,0.008671,0.002795,0.014180,0.004926,0.005396
2020-09-25,0.000970,0.070864,NaN,0.005008,0.014783,0.007254,0.001241,0.000401,0.001814,NaN,...,0.014551,0.007296,0.005199,7.852993e-18,0.000004,0.008032,0.006205,0.017761,0.006020,0.002698


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin       Barbour      Bibb    Blount   
2020-09-23      0.010587  0.001184  2.436691e-12  0.003689  0.036937   
2020-09-24      0.005293  0.010796  1.218346e-12  0.001845  0.018468   
2020-09-25      0.002647  0.005398  6.091728e-13  0.000922  0.009234   

Province_State                                                    ...  \
Admin2           Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2020-09-23      0.000300  0.013263  0.007498  0.000195 -0.029309  ...   
2020-09-24      0.000150  0.006632  0.016249  0.000098 -0.014654  ...   
2020-09-25      0.000075  0.003316  0.020320  0.000049 -0.073994  ...   

Province_State   Wyoming                                                   \
Admin2              Park Platte  Sheridan Sublette Sweetwater Teton Uinta   
2020-09-23      0.007813    0.0  0.011719      0.0        0.0   0.0   0.0   
2020-09-24      0.003906    0.0  0.005859      0.0        0.0   0.0   0.0   
2020-09-25      0.001953    0.0  0.002930      0.0        0.0   0.0   0.0   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-09-23      -0.999969      0.0    NaN  
2020-09-24      -0.999969      0.0    NaN  
2020-09-25      -0.999969      0.0    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200925,AK,8202,433130,NaN,441332,43.0,NaN,NaN,NaN,...,441332,6,0,0271fcc794329b2dd035402cb74aad56124c446b,0,0,0,0,0,NaN
1,20200925,AL,150658,963364,NaN,1097595,718.0,16852.0,NaN,1782.0,...,1114022,-15,74,08184faa52c95520b2871a34cbea8adad1c1d054,0,0,0,0,0,NaN
2,20200925,AR,79946,848822,NaN,926294,478.0,5202.0,224.0,NaN,...,928768,20,42,6d09ffffc7c9131f7489f7d8e3e4501ce55b8e1e,0,0,0,0,0,NaN
3,20200925,AS,0,1571,NaN,1571,NaN,NaN,NaN,NaN,...,1571,0,0,84c6296997b54d1474da9d9c51e33d48fda7fb5b,0,0,0,0,0,NaN
4,20200925,AZ,216367,1211657,NaN,1423603,521.0,21972.0,119.0,NaN,...,1428024,28,30,a7a38e5509bf8d1ebfe03d1d9acd5febad3546d0,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200925,AK,8202.0,433130.0,NaN,441332.0,43.0,NaN,NaN,NaN,...,441332,6,0,0271fcc794329b2dd035402cb74aad56124c446b,0,0,0,0,0,NaN
1,20200925,AL,150658.0,963364.0,NaN,1097595.0,718.0,16852.0,NaN,1782.0,...,1114022,-15,74,08184faa52c95520b2871a34cbea8adad1c1d054,0,0,0,0,0,NaN
2,20200925,AR,79946.0,848822.0,NaN,926294.0,478.0,5202.0,224.0,NaN,...,928768,20,42,6d09ffffc7c9131f7489f7d8e3e4501ce55b8e1e,0,0,0,0,0,NaN
3,20200925,AS,0.0,1571.0,NaN,1571.0,NaN,NaN,NaN,NaN,...,1571,0,0,84c6296997b54d1474da9d9c51e33d48fda7fb5b,0,0,0,0,0,NaN
4,20200925,AZ,216367.0,1211657.0,NaN,1423603.0,521.0,21972.0,119.0,NaN,...,1428024,28,30,a7a38e5509bf8d1ebfe03d1d9acd5febad3546d0,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-09-25,AK,8202,433130,NaN,441332,43.0,NaN,NaN,NaN,14.0,...,441332,6,0,0271fcc794329b2dd035402cb74aad56124c446b,0,0,0,0,0,NaN
2020-09-25,AL,150658,963364,NaN,1097595,718.0,16852.0,NaN,1782.0,NaN,...,1114022,-15,74,08184faa52c95520b2871a34cbea8adad1c1d054,0,0,0,0,0,NaN
2020-09-25,AR,79946,848822,NaN,926294,478.0,5202.0,224.0,NaN,95.0,...,928768,20,42,6d09ffffc7c9131f7489f7d8e3e4501ce55b8e1e,0,0,0,0,0,NaN
2020-09-25,AS,0,1571,NaN,1571,NaN,NaN,NaN,NaN,NaN,...,1571,0,0,84c6296997b54d1474da9d9c51e33d48fda7fb5b,0,0,0,0,0,NaN
2020-09-25,AZ,216367,1211657,NaN,1423603,521.0,21972.0,119.0,NaN,65.0,...,1428024,28,30,a7a38e5509bf8d1ebfe03d1d9acd5febad3546d0,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-09-25,AK,8202.0,433130.0,NaN,441332.0,43.0,NaN,NaN,NaN,14.0,...,441332,6,0,0271fcc794329b2dd035402cb74aad56124c446b,0,0,0,0,0,NaN
2020-09-25,AL,150658.0,963364.0,NaN,1097595.0,718.0,16852.0,NaN,1782.0,NaN,...,1114022,-15,74,08184faa52c95520b2871a34cbea8adad1c1d054,0,0,0,0,0,NaN
2020-09-25,AR,79946.0,848822.0,NaN,926294.0,478.0,5202.0,224.0,NaN,95.0,...,928768,20,42,6d09ffffc7c9131f7489f7d8e3e4501ce55b8e1e,0,0,0,0,0,NaN
2020-09-25,AS,0.0,1571.0,NaN,1571.0,NaN,NaN,NaN,NaN,NaN,...,1571,0,0,84c6296997b54d1474da9d9c51e33d48fda7fb5b,0,0,0,0,0,NaN
2020-09-25,AZ,216367.0,1211657.0,NaN,1423603.0,521.0,21972.0,119.0,NaN,65.0,...,1428024,28,30,a7a38e5509bf8d1ebfe03d1d9acd5febad3546d0,0,0,0,0,0,NaN
